# Born functions in SymPy and HKF

In [ ]:
import sympy as sym
sym.init_printing()

Define the Born function: $B = - \frac{1}{\varepsilon }$  
Define: $Q = \frac{1}{\varepsilon }{\left( {\frac{{\partial \ln \varepsilon }}{{\partial P}}} \right)_T}$  and show that $Q = \frac{{\partial B}}{{\partial P}}$  
Define: $Y = \frac{1}{\varepsilon }{\left( {\frac{{\partial \ln \varepsilon }}{{\partial T}}} \right)_P}$ and show that $Y = \frac{{\partial B}}{{\partial T}}$  
Define: $X = \frac{1}{\varepsilon }\left[ {{{\left( {\frac{{{\partial ^2}\ln \varepsilon }}{{\partial {T^2}}}} \right)}_P} - \left( {\frac{{\partial \ln \varepsilon }}{{\partial T}}} \right)_P^2} \right]$ and show that $X = \frac{{{\partial ^2}B}}{{\partial {T^2}}}$  
Define: $N = \frac{1}{\varepsilon }\left[ {{{\left( {\frac{{{\partial ^2}\ln \varepsilon }}{{\partial {P^2}}}} \right)}_T} - \left( {\frac{{\partial \ln \varepsilon }}{{\partial P}}} \right)_T^2} \right]$ and show that $N = \frac{{{\partial ^2}B}}{{\partial {P^2}}}$  
Define: $U = \frac{1}{\varepsilon }\left[ {\left( {\frac{{{\partial ^2}\ln \varepsilon }}{{\partial T\partial P}}} \right) - {{\left( {\frac{{\partial \ln \varepsilon }}{{\partial T}}} \right)}_P}{{\left( {\frac{{\partial \ln \varepsilon }}{{\partial P}}} \right)}_T}} \right]$   and show that $U = \frac{{{\partial ^2}B}}{{\partial T\partial P}}$

SymPy special function classes

In [ ]:
class X(sym.Function):
    nargs = (1, 2)

class N(sym.Function):
    nargs = (1, 2)

class U(sym.Function):
    nargs = (1, 2)

class Y(sym.Function):
    nargs = (1, 2)
    
    def fdiff(self, argindex=1):
        T,P = self.args
        if argindex == 1:
            return X(T,P)
        elif argindex == 2:
            return U(T,P)
        raise ArgumentIndexError(self, argindex)

class Q(sym.Function):
    nargs = (1, 2)
    
    def fdiff(self, argindex=1):
        T,P = self.args
        if argindex == 1:
            return U(T,P)
        elif argindex == 2:
            return N(T,P)
        raise ArgumentIndexError(self, argindex)
    
class B(sym.Function):
    nargs = (1, 2)
    
    def fdiff(self, argindex=1):
        T,P = self.args
        if argindex == 1:
            return Y(T,P)
        elif argindex == 2:
            return Q(T,P)
        raise ArgumentIndexError(self, argindex)

In [ ]:
from sympy.printing.ccode import C99CodePrinter
printer = C99CodePrinter(settings={"user_functions":{"B":"B", "Q":"Q","Y":"Y","N":"N","X":"X","U":"U"}})

## HKF - Part III

${\bar v^o} = {a_1} + {a_2}P + \frac{{\left( {{a_3} + {a_4}P} \right)T}}{{T - \theta }} - \omega Q$  
$\frac{{\partial {{\bar v}^o}}}{{\partial T}} =  - \frac{{\partial {{\bar s}^o}}}{{\partial P}}$  
$\bar C_{{P_r}}^o = {c_1} + \frac{{{c_2}}}{{{{\left( {T - \theta } \right)}^2}}}$

In [ ]:
a1,a2,a3,a4 = sym.symbols('a_1 a_2 a_3 a_4')
c1,c2 = sym.symbols('c_1 c_2')
omega,theta = sym.symbols('omega theta')
T,P,Tr,Pr = sym.symbols('T P T_r P_r')

In [ ]:
vtp = a1 + a2*P + (a3 + a4*P)*T/(T-theta) -omega*Q(T,P)
cpr = c1 + c2/(T-theta)**2
vtp,cpr

In [ ]:
Href,Sref = sym.symbols('H_ref S_ref')

In [ ]:
spr = Sref + sym.integrate(cpr/T,(T,Tr,T))
spr

In [ ]:
gtp = sym.integrate(spr, (T,Tr,T))

In [ ]:
gtp += sym.integrate(a1 + a2*P + (a3 + a4*P)*T/(T-theta),(P,Pr,P)) - omega*(B(T,P) - B(T,Pr))

In [ ]:
gtp

In [ ]:
T*gtp.diff(T,T).subs(P,Pr).simplify().collect(c1)